# Prototypisches Data Profiling für die Datensätze der [Food Security Portal API](http://www.foodsecurityportal.org/api)

Die Qualität verfügbarer Open Data Datensätze ist häufig sehr unterschiedlich und die Recherche nach hochwertigen Datensätzen mühsam. <br>
Dieses prototypische Dashboard soll mit Hilfe eines Datenprofils Methoden darstellen, die Suche nach nützlichen und aussagekräftigen Open Data Datensätzen zu vereinfachen. <br>
Dafür kann sich für die Datensätze aus dem Food Security Portal Data Dashboard von IFPRI jeweils ein Datensatz- und Spaltenprofil angezeigt werden lassen.

In [27]:
import pandas as pd
import ipywidgets as widgets
import webbrowser
from IPython.display import display, Markdown
from traitlets import traitlets

In [28]:
#create Logger
import logging
import sys
logger = logging.getLogger()
logging.basicConfig(format='%(levelname)s : %(message)s',level=logging.INFO, stream=sys.stdout)

In [29]:
#Style Optionen
pd.set_option('display.max_colwidth', -1) #alle Daten in einer Spalte ausgeben
pd.set_option('precision',4) #nur 3 Nachkommastellen anzeigen
#turn extracted text into html linebreak format, source: 
pd.DataFrame.base_to_html = pd.DataFrame.to_html
pd.DataFrame.to_html = (
    lambda df, *args, **kwargs: 
        (df.base_to_html(*args, **kwargs)
           .replace(r"\n", "<br/>"))
)

In [30]:
#Import der Datenstruktur
import dataset_profile
import column_profile
import import_data

In [31]:
# Datensatz-Links
dataset_links = import_data.collect_all_dataset_links_fsp()
# Metadaten-Links
metadata_links = import_data.collect_all_metadata_links_fsp()

In [32]:
# Matching der Datenbanktitel mit Weltbank-Indikatoren
indicator_match_fsp_wb = {'Agricultural Land (% of land area)':'AG.LND.AGRI.ZS', 'Agriculture, Value Added (% of GDP)':'NV.AGR.TOTL.ZS', 
                     'External Debt (% of GNI)':'DT.DOD.DECT.GN.ZS', 'Foreign Direct Investment (Current $US)':'BX.KLT.DINV.CD.WD',
                     'GDP (current $US)':'NY.GDP.MKTP.CD', 'Global Inflation Dataset':'NY.GDP.DEFL.KD.ZG', 
                    'GNI per Capita (current $US)':'NY.GNP.PCAP.CD', 'National Poverty Rates (%)':'SI.POV.NAHC', 
                         'Population Density':'EN.POP.DNST', 'Under 5 Mortality Rate':'SH.DYN.MORT'}

In [33]:
#Dictionary für globale Variablen
profiling_data = {'dataframe_fsp': None, 'metadata_fsp' : None, 'dataframe_wb' : None, 'metadata_wb' : None}

In [70]:
# Definition der strukturierten Ausgabe der Metadaten und des Datenprofils
def print_ds_output(d,m):
    print("\nVerfügbare Metadaten:\n")
    if all('N/A' in x for x in m.values()):
        logging.info("keine Metadaten in API vorhanden")
    metadata = pd.DataFrame.from_dict(m, orient='index', columns=["Metadaten"])
    metadata.index.name = "Kriterien"
    metadata.sort_index(inplace=True)
    display(metadata)
    print("\nDatenprofil zu Datensatz:\n")
    profile = dataset_profile.describe_ds_as_dataframe(d, m)
    display(profile)

In [71]:
ds_title = widgets.Dropdown(
    options=dataset_links.keys(),
    description='Datensätze:',
    value = None,
    disabled=False,
    layout=widgets.Layout(width='33%')
)

In [72]:
#download datasets, wenn in Dropdown-Menü ausgewählt
def dropdown_observe(v):
    profiling_data['dataframe_fsp'] = import_data.get_dataset_fsp(dataset_links[v['new']])
    profiling_data['metadata_fsp'] = import_data.get_metadata_attributes_fsp(metadata_links.get(v['new'], 'N/A'))

In [73]:
ds_title.observe(dropdown_observe, names='value')

### Datensatzprofil

Über ein Dropdown Menü kann einer der zur Verfügung stehenden Datensatz zum Profiling ausgewählt werden. Dieser wird dann online abgerufen und zwischengespeichert.
Über den Button kann das Datensatzprofil für den jeweiligen Datensatz abgerufen werden. Es werden die entsprechenden Metadaten und das Datensatzprofil dargestellt. <br>

#### Vergleichsquelle: World Bank Datensatzprofil
Die meisten Datensätze in der Datenbank des Food Security Portals sind Sekundärquellen, die ursprünglich z.B. von der FAO oder Weltbank herausgegeben werden. Zur Überprüfung der Aussagekraft des Datensatzes kann unter anderem ein Vergleich zwischen verschiedenen Versionen der Quelle dienen. 
Daher wird hier als prototypisches Beispiel bei Abruf eines Datenprofils, wenn vorhanden, der entsprechende Datensatz und die dazugehörigen Metadaten aus der Weltbank-API abgerufen und als eigenes vergleichendes Datenprofil dargestellt.

In [74]:
#create button that can return value (source: )
class ValueButton(widgets.Button):
    def __init__(self, value=None, *args, **kwargs):
        super(ValueButton, self).__init__(*args, **kwargs)
        # Create a value attribute.
        self.add_traits(value=traitlets.Any(value))

In [75]:
#create buttons for dataset profiling
button_dataprofile = ValueButton(description="Datenprofil", layout=widgets.Layout(width='33%'))
output_title = widgets.Output()
output_fsp = widgets.Output()
output_wb = widgets.Output()

In [76]:
#create structure for output of fsp data and download and output worldbank data if available on button click
def on_button_dataprofile_clicked(df_fsp):
    output_title.clear_output()
    output_fsp.clear_output()
    output_wb.clear_output()
    with output_title:
        title = widgets.HTML(value="<h3>Datensatz: "+ds_title.value+"</h3>")
        display(title)
    with output_fsp:
        print("Ursprung: Food Security Portal")
        print_ds_output(profiling_data['dataframe_fsp'],profiling_data['metadata_fsp'])
    with output_wb:
        if (ds_title.value in indicator_match_fsp_wb.keys()):
            print("Vergleichsquelle wird geladen (Ursprung: World Bank)")
            profiling_data['dataframe_wb'] = import_data.get_dataset_wb(indicator_match_fsp_wb[ds_title.value])
            profiling_data['metadata_wb'] = import_data.get_metadata_attributes_wb(indicator_match_fsp_wb[ds_title.value])
            print_ds_output(profiling_data['dataframe_wb'], profiling_data['metadata_wb'])
        else:
            print("Keine Vergleichsquelle vorhanden")
        df_fsp.value = profiling_data['dataframe_fsp'].columns
                         

In [77]:
#bind button to click method
button_dataprofile.on_click(on_button_dataprofile_clicked)

In [78]:
#create column selection that gets value from chosen dataset (after button click)
column = widgets.Select(
    options= "",
    rows=10,
    description='Spalten:',
    disabled=False,
    layout=widgets.Layout(width='33%')
)

In [79]:
def set_column_options(_):
    column.options = button_dataprofile.value

In [80]:
button_dataprofile.observe(set_column_options)

In [81]:
#create column buttons and output structure
button_columnprofile = ValueButton(description="Spaltenprofil", layout=widgets.Layout(width='33%'))
output_columnprofile_fsp = widgets.Output()
output_columnprofile_wb = widgets.Output()

In [82]:
def print_columnprofile_output(source='fsp'):
    print("Spalte: ",column.value)
    print("\nSpaltenprofil zu Datensatz:")
    if source=='wb':
        display(column_profile.describe_dc_as_dataframe(profiling_data['dataframe_wb'][column.value], profiling_data['metadata_wb']))
    else:
        display(column_profile.describe_dc_as_dataframe(profiling_data['dataframe_fsp'][column.value], profiling_data['metadata_fsp']))

In [83]:
#define output of column profile on button click
def on_button_columnprofile_clicked(_):
    output_columnprofile_fsp.clear_output()
    output_columnprofile_wb.clear_output()
    with output_columnprofile_fsp:
        print_columnprofile_output()
    with output_columnprofile_wb:
        if (ds_title.value in indicator_match_fsp_wb.keys()):
            if(column.value in profiling_data['dataframe_wb'].keys()):
                print_columnprofile_output('wb')
            else:
                print("Spalte nicht vorhanden")
        else:
            print("Keine Vergleichsquelle vorhanden")

In [84]:
#bind column button on click method
button_columnprofile.on_click(on_button_columnprofile_clicked)

In [85]:
#Quelldatei herunterladen
button_open_file = widgets.Button(description = "Quelldatei herunterladen (foodsecurityportal.org)", layout=widgets.Layout(width='33%'))

def on_button_open_file_clicked(_):
    # generate an URL
    url = dataset_links[ds_title.value]
    webbrowser.open_new_tab(url)

button_open_file.on_click(on_button_open_file_clicked)

In [86]:
# create display
display(widgets.HBox([ds_title, button_dataprofile, button_open_file]))
display(widgets.HBox([output_title]))
display(widgets.HBox([widgets.VBox([output_fsp], layout=widgets.Layout(width='50%')), widgets.VBox([output_wb], layout=widgets.Layout(width='50%'))]))

In [87]:
display(widgets.HBox([column, button_columnprofile]))
display(widgets.HBox([widgets.VBox([output_columnprofile_fsp], layout=widgets.Layout(width='50%')), widgets.VBox([output_columnprofile_wb], layout=widgets.Layout(width='50%'))]))

### Datensatzprofil: verfügbare Attribute

 * **Domain**: als Domain angegebene Kategorisierung der enthaltenen Spalten
 * **geographischer Geltungsbereich**: geographischer Geltungsbereich des Datensatzes. Ausgegeben werden entweder Kontinente oder "Welt", wenn alle Kontinente enthalten sind.
 * **Datengröße in KB**: gibt die Größe der intern verarbeiteten Datei an als Richtwert für den Dateidownload; Downloadgröße kann geringer sein
 * **Zeitraum**: der verfügbare Zeitrahmen mit min. einem Datenpunkt im Datensatz
 * **Zeitliche Granularität**: Überprüfung des tatsächlich verfügbaren Intervalls der enthaltenen Datenpunkte (z.B. täglich, wöchentlich, ...)
 * **Anzahl der Zeilen**: Anzahl der Zeilen im Datensatz (meist Zeitreihen: Anzahl an Zeitpunkten)
 * **Anzahl der Spalten**: Anzahl der Spalten im Datensatz (meist Länder: Anzahl an Ländern)
 * **Distinkte Werte (Prozent)**: Prozentanteil an Werten, die einzigartig im gesamten Datensatz sind
 * **Fehlende Werte (Prozent)**: Prozentanteil an fehlenden Werten im gesamten Datensatz
 * **Spalten ohne Werte (n)**: Anzahl an Spalten, die gar keine Werte enthalten
 * **Datentypen**: Überprüfung und Kategorisierung der enthaltenen Datentypen in numerisch (NUM), textuell (STRING) oder konstant (CONST)
 * **Open Data Schema**:  Kategorisierung der Datenquelle in Stufe 1-5 nach dem 5 Star Open Data Schema (https://5stardata.info/de/)
 * **Spalten mit exakt selben Werten**: Überprüfung, ob duplizierte Spalten existieren und Ausgabe der Spalten mit exakt selben Werten
 * **Aggregations-Check**: Unter der Annahme, dass die "Welt"-Spalte, wenn vorhanden, eine Aggregationsspalte darstellt, wird überprüft, ob die Aggregation mit dem Durchschnitt oder der Summe der restlichen Spalten übereinstimmt; Ausgabe der nicht übereinstimmenden Indizes, Ausgabe "alle Zeilen", wenn in keiner Zeile eine Übereinstimmung existiert
 * **Überprüfung des Wertebereichs**: Überprüfung, ob alle Werte dem vorgegebenen Wertebereich der angegebenen Einheit entsprechen; Ausgabe der Spalten, in denen mindestens ein Wert den Wertebereich verletzt
 * **Herausgeber-Kategorie"**: Kategorisierung des Herausgebers der Quelle in Oberkategorie (z.B. FAO -> Zwischenstaatliche Organisation)
 * **Domain-Check**: Überprüfung, ob alle Spalten der angegebenen Domain ensprechen, wenn diese "Country" ist und Ausgabe der Länder, die nicht als Land erkannt wurden (mit ihrer eigentlichen Domain, wenn möglich)
 * **Verzögerung Veröffentlichung in Monaten**: Unterschied zwischen angegebenem Veröffentlichungsdatum (Erstellungdatum in den Metadaten) und dem letzten Zeitpunkt in den Daten in Monaten 

### Spaltenprofil: verfügbare Attribute
* **Metadaten spezifisch für Spalte** : prototypische Extraktion der Metadaten assoziert zum Spaltentitel, wenn möglich und vorhanden
* **Anzahl an Zeilen** : Anzahl der Zeilen mit Werten
* **Anzahl an fehlenden Werten** : Anzahl der Zeilen ohne Werte
* **Fehlende Werte (Prozent)** : Prozentanteil der Zeilen in der Spalte, die keine Werte darstellen
* **Distinkte Werte (Prozent)** : Prozentanteil der Werte in der Spalte, die nur einmal vorkommen
* **Konstanz (Prozent)** : Konstanz definiert als Anzahl des häufigsten Wertes dividiert durch die Anzahl der Werte in der Spalte
* **Datenpunkte vorhanden für** : konsekutive Zeiträume im Datensatz, in denen mindestens ein Wert vorhanden ist